# Regression with an Insurance Dataset

## Modified the metric from `log1p RMSE` to Raw `RMSE` for both Keras and Keras Tuner

## Keras - Attempt 1

## Attempt 1

In [ ]:
# Reproducibility
import os, random, time, json
import numpy as np
import pandas as pd
random.seed(42)
np.random.seed(42)
import tensorflow as tf
from sklearn.utils import all_estimators
# scikit-learn has no global seed; we set seeds in train_test_split
tf.random.set_seed(42)

# Imports
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import RootMeanSquaredError, MeanAbsoluteError

# Load Data
df_train = pd.read_csv('playground-series-s4e12/train.csv')
df_test = pd.read_csv('playground-series-s4e12/test.csv')
# Identify id and target columns
id_col = 'id'
target_columns = ['Premium Amount']

# Combine and encode target
df = df_train.copy()
y_values = df[target_columns].astype(float).values
# Use log1p if all non-negative
if np.all(y_values >= 0):
    y_enc = np.log1p(y_values)
else:
    y_enc = y_values

# Prepare features
drop_cols = [id_col] + target_columns
X = df.drop(columns=drop_cols, errors='ignore')
# Testing set
X_test = df_test.drop(columns=drop_cols, errors='ignore')
test_ids = df_test[id_col]

# Feature engineering: drop cols with all missing
non_null_cols = X.columns[X.notna().any()].tolist()
X = X[non_null_cols]
X_test = X_test[non_null_cols]
# Identify categorical vs numeric
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
# Drop high-cardinality categorical cols
high_card = [c for c in categorical_cols if X[c].nunique() > 50]
for c in high_card:
    X.drop(columns=c, inplace=True)
    X_test.drop(columns=c, inplace=True)
categorical_cols = [c for c in categorical_cols if c not in high_card]
numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()

# Preprocessing pipeline
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ],
    remainder='drop'
)

# Fit and transform
df_train_proc = preprocessor.fit_transform(X)
df_test_proc = preprocessor.transform(X_test)
n_samples, n_features = df_train_proc.shape

# Build model (small dataset branch)
from tensorflow.keras import layers, models, optimizers
hidden_units = [min(n_features*2, 128), min(n_features, 64)]
model = models.Sequential()
for units in hidden_units:
    model.add(layers.Dense(units, activation='relu'))
    model.add(layers.Dropout(0.3))
model.add(layers.Dense(1, activation='linear'))

def mse_real(y_true_log, y_pred_log):
    y_true = tf.math.expm1(y_true_log)
    y_pred = tf.math.expm1(y_pred_log)
    return tf.reduce_mean(tf.square(y_true - y_pred))
mse_real.__name__ = 'mse_real'      

def rmse_real(y_true_log, y_pred_log):
    y_true = tf.math.expm1(y_true_log)
    y_pred = tf.math.expm1(y_pred_log)
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))
rmse_real.__name__ = 'rmse_real'

# Compile
model.compile(
    optimizer=optimizers.Adam(),
    loss='mean_squared_error',
    metrics=[rmse_real, mse_real]
)

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)
]

# Training
time_start = time.time()
history = model.fit(
    df_train_proc, y_enc,
    validation_split=0.2,
    epochs=100,
    batch_size=64,
    callbacks=callbacks,
    verbose=2
)
time_duration = time.time() - time_start

# Save results
results = {
    'training_loss': history.history['mse_real'][-1],
    'validation_loss': history.history['val_mse_real'][-1],
    'training_RMSE': history.history['rmse_real'][-1],
    'validation_RMSE': history.history['val_rmse_real'][-1]
}
with open('Keras/results.json', 'w') as f:
    json.dump(results, f)

# Prediction & Submission
raw_preds = model.predict(df_test_proc)
# Reverse log
final = np.expm1(np.clip(raw_preds, a_min=None, a_max=20))
# Ensure 2D
if final.ndim == 1:
    final = final.reshape(-1,1)
submission = pd.DataFrame(final, columns=target_columns)
submission.insert(0, id_col, test_ids.reset_index(drop=True))
submission.to_csv('Keras/submission_result.csv', index=False)

2025-07-09 21:42:42.139689: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Epoch 1/100


15000/15000 - 31s - 2ms/step - loss: 1.7048 - mse_real: 4287889.0000 - rmse_real: 1247.4364 - val_loss: 1.1586 - val_mse_real: 876568.3125 - val_rmse_real: 927.6188
Epoch 2/100


15000/15000 - 31s - 2ms/step - loss: 1.1823 - mse_real: 890864.3125 - rmse_real: 934.9928 - val_loss: 1.1495 - val_mse_real: 853542.0625 - val_rmse_real: 915.2062
Epoch 3/100


15000/15000 - 30s - 2ms/step - loss: 1.1640 - mse_real: 879367.4375 - rmse_real: 928.8301 - val_loss: 1.1468 - val_mse_real: 858588.0625 - val_rmse_real: 917.9031
Epoch 4/100
15000/15000 - 30s - 2ms/step - loss: 1.1606 - mse_real: 877315.8750 - rmse_real: 927.7182 - val_loss: 1.1469 - val_mse_real: 860355.8750 - val_rmse_real: 918.8455
Epoch 5/100


...

15000/15000 - 22s - 1ms/step - loss: 1.1557 - mse_real: 875998.1250 - rmse_real: 926.9874 - val_loss: 1.1428 - val_mse_real: 861422.1250 - val_rmse_real: 919.3741
Epoch 29/100
15000/15000 - 21s - 1ms/step - loss: 1.1555 - mse_real: 876012.5625 - rmse_real: 926.9996 - val_loss: 1.1430 - val_mse_real: 859999.4375 - val_rmse_real: 918.6033
Epoch 30/100
15000/15000 - 22s - 1ms/step - loss: 1.1555 - mse_real: 876289.0000 - rmse_real: 927.1256 - val_loss: 1.1430 - val_mse_real: 855842.6250 - val_rmse_real: 916.3820
Epoch 31/100


15000/15000 - 24s - 2ms/step - loss: 1.1555 - mse_real: 876067.6875 - rmse_real: 927.0204 - val_loss: 1.1425 - val_mse_real: 862858.5000 - val_rmse_real: 920.1399
Epoch 32/100


15000/15000 - 25s - 2ms/step - loss: 1.1556 - mse_real: 876101.2500 - rmse_real: 927.0367 - val_loss: 1.1424 - val_mse_real: 859593.6875 - val_rmse_real: 918.3905
Epoch 33/100
15000/15000 - 27s - 2ms/step - loss: 1.1554 - mse_real: 876106.5000 - rmse_real: 927.0338 - val_loss: 1.1425 - val_mse_real: 861296.0000 - val_rmse_real: 919.2961
Epoch 34/100
15000/15000 - 27s - 2ms/step - loss: 1.1551 - mse_real: 875949.7500 - rmse_real: 926.9487 - val_loss: 1.1427 - val_mse_real: 859829.6875 - val_rmse_real: 918.5179
Epoch 35/100


15000/15000 - 24s - 2ms/step - loss: 1.1553 - mse_real: 875999.1875 - rmse_real: 926.9731 - val_loss: 1.1423 - val_mse_real: 861645.5000 - val_rmse_real: 919.4887
Epoch 36/100


15000/15000 - 22s - 1ms/step - loss: 1.1557 - mse_real: 876147.1875 - rmse_real: 927.0469 - val_loss: 1.1421 - val_mse_real: 855427.9375 - val_rmse_real: 916.1270
Epoch 37/100
15000/15000 - 23s - 2ms/step - loss: 1.1553 - mse_real: 875734.8125 - rmse_real: 926.8301 - val_loss: 1.1423 - val_mse_real: 862999.7500 - val_rmse_real: 920.2047
Epoch 38/100
15000/15000 - 23s - 2ms/step - loss: 1.1550 - mse_real: 875649.1875 - rmse_real: 926.7905 - val_loss: 1.1430 - val_mse_real: 859685.2500 - val_rmse_real: 918.4477
Epoch 39/100
15000/15000 - 25s - 2ms/step - loss: 1.1550 - mse_real: 875692.3125 - rmse_real: 926.8151 - val_loss: 1.1435 - val_mse_real: 859122.1875 - val_rmse_real: 918.1373
Epoch 40/100
15000/15000 - 23s - 2ms/step - loss: 1.1551 - mse_real: 875879.5000 - rmse_real: 926.9092 - val_loss: 1.1440 - val_mse_real: 857177.8125 - val_rmse_real: 917.1180
Epoch 41/100
15000/15000 - 25s - 2ms/step - loss: 1.1551 - mse_real: 875772.6250 - rmse_real: 926.8583 - val_loss: 1.1426 - val_mse_r

# Keras Tuner - 1 Attempt

# Attempt 1

In [ ]:
# Reproducibility
import os, random, time, json
import numpy as np
import pandas as pd
random.seed(42)
np.random.seed(42)
import tensorflow as tf
from sklearn.utils import all_estimators
# scikit-learn has no global seed; we set seeds in train_test_split
tf.random.set_seed(42)

# Imports
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import RootMeanSquaredError, MeanAbsoluteError

# Load Data
df_train = pd.read_csv('playground-series-s4e12/train.csv')
df_test = pd.read_csv('playground-series-s4e12/test.csv')
# Identify id and target columns
id_col = 'id'
target_columns = ['Premium Amount']

# Combine and encode target
df = df_train.copy()
y_values = df[target_columns].astype(float).values
# Use log1p if all non-negative
if np.all(y_values >= 0):
    y_enc = np.log1p(y_values)
else:
    y_enc = y_values

# Prepare features
drop_cols = [id_col] + target_columns
X = df.drop(columns=drop_cols, errors='ignore')
# Testing set
X_test = df_test.drop(columns=drop_cols, errors='ignore')
test_ids = df_test[id_col]

# Feature engineering: drop cols with all missing
non_null_cols = X.columns[X.notna().any()].tolist()
X = X[non_null_cols]
X_test = X_test[non_null_cols]
# Identify categorical vs numeric
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
# Drop high-cardinality categorical cols
high_card = [c for c in categorical_cols if X[c].nunique() > 50]
for c in high_card:
    X.drop(columns=c, inplace=True)
    X_test.drop(columns=c, inplace=True)
categorical_cols = [c for c in categorical_cols if c not in high_card]
numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()

# Preprocessing pipeline
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ],
    remainder='drop'
)

# Fit and transform
df_train_proc = preprocessor.fit_transform(X)
df_test_proc = preprocessor.transform(X_test)
n_samples, n_features = df_train_proc.shape

# Build model (Keras-Tuner snippet)
import keras_tuner as kt
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import optimizers
from tensorflow.keras.metrics import RootMeanSquaredError, MeanAbsoluteError

# Define early stopping and checkpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

# Preserve input dim
n_features = df_train_proc.shape[1]


def mse_real(y_true_log, y_pred_log):
    y_true = tf.math.expm1(y_true_log)
    y_pred = tf.math.expm1(y_pred_log)
    return tf.reduce_mean(tf.square(y_true - y_pred))
mse_real.__name__ = 'mse_real'      

def rmse_real(y_true_log, y_pred_log):
    y_true = tf.math.expm1(y_true_log)
    y_pred = tf.math.expm1(y_pred_log)
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))
rmse_real.__name__ = 'rmse_real'


class MyHyperModel(kt.HyperModel):
    def build(self, hp):
        layers = hp.Int('layers', 2, 8)
        units = hp.Int('units', 64, 1024)
        act = hp.Choice('activation', ['relu'])
        drop = hp.Float('dropout', 0.0, 0.5)
        opt = hp.Choice('optimizer', ['adam'])
        lr = hp.Float('learning_rate', 1e-05, 0.01, sampling='log')

        inputs = Input(shape=(n_features,))
        x = inputs
        for _ in range(layers):
            x = Dense(units, activation=act)(x)
            x = Dropout(drop)(x)
        x = Dense(1, activation='linear')(x)
        model = Model(inputs, x)
        model.compile(optimizer=opt, loss='mean_squared_error', metrics=[rmse_real, mse_real])
        return model

# Tuner setup
bs = 64  # batch size
ep = 100  # epochs

tuner = kt.BayesianOptimization(
    MyHyperModel(),
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    seed=42,
    overwrite=False,
    project_name='bayesian_tuner'
)

if y_enc is not None:
    tuner.search(
        df_train_proc, y_enc,
        validation_split=0.2,
        batch_size=bs, epochs=ep,
        callbacks=[early_stopping, checkpoint]
    )
else:
    tuner.search(
        df_train_proc, y_enc,
        validation_split=0.2,
        batch_size=bs, epochs=ep,
        callbacks=[early_stopping, checkpoint]
    )

model = tuner.hypermodel.build(
    tuner.get_best_hyperparameters(1)[0]
)

time_start = time.time()

# Retrain model with original callbacks and data
if y_enc is not None:
    history = model.fit(
        df_train_proc, y_enc,
        validation_split=0.2,
        epochs=100, batch_size=bs,
        callbacks=[early_stopping, checkpoint],
        verbose=2
    )
else:
    history = model.fit(
        df_train_proc, y_enc,
        validation_split=0.2,
        epochs=100, batch_size=bs,
        callbacks=[early_stopping, checkpoint],
        verbose=2
    )

time_duration = time.time() - time_start

# Save results
results = {
    'training_loss': history.history['mse_real'][-1],
    'validation_loss': history.history['val_mse_real'][-1],
    'training_RMSE': history.history['rmse_real'][-1],
    'validation_RMSE': history.history['val_rmse_real'][-1]
}
with open('results.json', 'w') as f:
    json.dump(results, f)

# Prediction & Submission
raw_preds = model.predict(df_test_proc)
# Reverse log
final = np.expm1(np.clip(raw_preds, a_min=None, a_max=20))
# Ensure 2D
if final.ndim == 1:
    final = final.reshape(-1,1)
submission = pd.DataFrame(final, columns=target_columns)
submission.insert(0, id_col, test_ids.reset_index(drop=True))
submission.to_csv('submission_result.csv', index=False)

Trial 10 Complete [00h 21m 06s]
val_loss: 1.1446022987365723

Best val_loss So Far: 1.1321594715118408
Total elapsed time: 04h 26m 48s
Epoch 1/100


2025-07-15 19:10:00.840071: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_42', 496 bytes spill stores, 496 bytes spill loads

2025-07-15 19:10:00.889840: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_42', 240 bytes spill stores, 240 bytes spill loads

2025-07-15 19:10:00.943026: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_42', 152 bytes spill stores, 152 bytes spill loads

2025-07-15 19:10:01.026897: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_42', 240 bytes spill stores, 240 bytes spill loads

2025-07-15 19:10:01.084641: I external/l

15000/15000 - 108s - 7ms/step - loss: 1.2427 - mse_real: 1356109.5000 - rmse_real: 951.8457 - val_loss: 1.1511 - val_mse_real: 877535.3125 - val_rmse_real: 928.0486
Epoch 2/100


15000/15000 - 93s - 6ms/step - loss: 1.1817 - mse_real: 885380.5625 - rmse_real: 931.9096 - val_loss: 1.1477 - val_mse_real: 879056.4375 - val_rmse_real: 928.8105
Epoch 3/100


15000/15000 - 79s - 5ms/step - loss: 1.1733 - mse_real: 881220.3750 - rmse_real: 929.7810 - val_loss: 1.1474 - val_mse_real: 867389.0000 - val_rmse_real: 922.6240
Epoch 4/100


...

15000/15000 - 124s - 8ms/step - loss: 1.1668 - mse_real: 879020.5000 - rmse_real: 928.5814 - val_loss: 1.1426 - val_mse_real: 867315.2500 - val_rmse_real: 922.5851
Epoch 6/100


15000/15000 - 116s - 8ms/step - loss: 1.1637 - mse_real: 877490.4375 - rmse_real: 927.7740 - val_loss: 1.1417 - val_mse_real: 869388.5000 - val_rmse_real: 923.6783
Epoch 7/100


15000/15000 - 119s - 8ms/step - loss: 1.1616 - mse_real: 876231.2500 - rmse_real: 927.1092 - val_loss: 1.1409 - val_mse_real: 867519.0625 - val_rmse_real: 922.6592
Epoch 8/100


15000/15000 - 130s - 9ms/step - loss: 1.1607 - mse_real: 877101.1875 - rmse_real: 927.1332 - val_loss: 1.1408 - val_mse_real: 863777.4375 - val_rmse_real: 920.6606
Epoch 9/100


15000/15000 - 118s - 8ms/step - loss: 1.1595 - mse_real: 875484.9375 - rmse_real: 926.7154 - val_loss: 1.1398 - val_mse_real: 866993.8125 - val_rmse_real: 922.3813
Epoch 10/100
15000/15000 - 116s - 8ms/step - loss: 1.1588 - mse_real: 875258.5625 - rmse_real: 926.5756 - val_loss: 1.1402 - val_mse_real: 866803.3750 - val_rmse_real: 922.2396
Epoch 11/100
15000/15000 - 99s - 7ms/step - loss: 1.1575 - mse_real: 874781.1250 - rmse_real: 926.3040 - val_loss: 1.1444 - val_mse_real: 858642.6875 - val_rmse_real: 917.9684
Epoch 12/100
15000/15000 - 95s - 6ms/step - loss: 1.1567 - mse_real: 874309.6875 - rmse_real: 926.0527 - val_loss: 1.1415 - val_mse_real: 861942.0625 - val_rmse_real: 919.7040
Epoch 13/100
15000/15000 - 96s - 6ms/step - loss: 1.1573 - mse_real: 269315424.0000 - rmse_real: 1059.4489 - val_loss: 1.1429 - val_mse_real: 859218.0000 - val_rmse_real: 918.2705
Epoch 14/100
15000/15000 - 96s - 6ms/step - loss: 1.1564 - mse_real: 881126.5625 - rmse_real: 926.5775 - val_loss: 1.1421 - val

In [ ]:
print(time_duration)

2688.784352
